<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/model_train_cigarette.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Train a model for sigarette detection

In [1]:
import torch
import torchvision
import psutil
import os
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from torch.utils.data import DataLoader, Dataset
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F, Compose, Resize, ToTensor, Normalize
from torchvision import transforms
from PIL import Image
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
class CigaretteDataset(Dataset):
    def __init__(self, coco_annotation_file, image_dir, transform=None):
        self.coco = COCO(coco_annotation_file)
        self.image_dir = image_dir
        self.transform = transform
        cat_ids = self.coco.getCatIds(catNms=['cigarette'])
        if not cat_ids:
            raise ValueError("Nessuna categoria 'cigarette' trovata nel file COCO.")
        self.image_ids = list(set(self.coco.getImgIds(catIds=cat_ids)))

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]
        image = Image.open(f"{self.image_dir}/{img_info['file_name']}").convert("RGB")

        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        annotations = self.coco.loadAnns(ann_ids)
        boxes, labels = [], []

        for ann in annotations:
            if ann['category_id'] in self.coco.getCatIds(catNms=['cigarette']):
                x, y, w, h = ann['bbox']
                boxes.append([x, y, x + w, y + h])
                labels.append(1)  # Classe 1 per 'cigarette'

        if len(boxes) == 0:
            return None

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        target = {'boxes': boxes, 'labels': labels, 'image_id': torch.tensor([img_id])}

        if self.transform:
            image = self.transform(image)
        return image, target

In [4]:
def collate_fn(batch):
    return tuple(zip(*[b for b in batch if b is not None]))

In [5]:
def check_system_usage():
    print(f"CPU Usage: {psutil.cpu_percent()}%")
    print(f"RAM Usage: {psutil.virtual_memory().percent}%")
    if torch.cuda.is_available():
        print(f"GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        print(f"GPU Memory Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

In [6]:
def evaluate_model(model, dataset, device):
    model.to(device)
    model.eval()

    coco_dt = []  # Initialize here

    for idx in tqdm(range(len(dataset)), desc="Evaluating Model"):
        image, target = dataset[idx]

        if not isinstance(image, torch.Tensor):  # Convert if still a PIL image
            image = transforms.ToTensor()(image)

        image = image.to(device)  # Move input to the same device as the model

        with torch.no_grad():
            prediction = model([image])

        # Extract predictions (boxes, labels, scores) and convert them into COCO format
        for box, score, label in zip(prediction[0]['boxes'], prediction[0]['scores'], prediction[0]['labels']):
            coco_dt.append({
                'image_id': target['image_id'].item(),
                'category_id': label.item(),
                'bbox': box.tolist(),
                'score': score.item()
            })

    return coco_dt  # Make sure to return the results

In [7]:
def train_model(dataset, num_epochs=10, val_dataset=None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = fasterrcnn_resnet50_fpn(weights='DEFAULT')
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes=2)
    model.to(device)

    data_loader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn, num_workers=2, pin_memory=True, persistent_workers=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
    scaler = torch.amp.GradScaler('cuda')
    epoch_losses = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        start_time = time.time()
        print(f"Inizio Epoca {epoch+1}/{num_epochs}")

        # Usa tqdm per aggiungere la progress bar
        for batch_idx, (images, targets) in enumerate(tqdm(data_loader, desc=f"Epoca {epoch+1}/{num_epochs}", ncols=100, unit="batch")):
            images = [transforms.ToTensor()(img).to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            optimizer.zero_grad()

            loss_dict = model(images, targets)
            loss = sum(loss for loss in loss_dict.values())

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()

        avg_loss = total_loss / len(data_loader)
        epoch_losses.append(avg_loss)

        print(f"Epoca {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Tempo: {time.time() - start_time:.2f}s")
        check_system_usage()

        # Valutazione del modello su dataset di validazione
        if val_dataset:
            evaluate_model(model, val_dataset, device)

    torch.save(model.state_dict(), "fasterrcnn_cigarette.pth")
    return model

In [8]:
if __name__ == "__main__":
    train_image_dir = '/content/drive/MyDrive/Photo/train'
    train_coco_annotation_file = '/content/drive/MyDrive/Photo/train/_annotations.coco.json'

    valid_image_dir = '/content/drive/MyDrive/Photo/valid'
    valid_coco_annotation_file = '/content/drive/MyDrive/Photo/valid/_annotations.coco.json'

    dataset = CigaretteDataset(train_coco_annotation_file, train_image_dir)
    val_dataset = CigaretteDataset(valid_coco_annotation_file, valid_image_dir)

    model = train_model(dataset, num_epochs=10, val_dataset= val_dataset)

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Inizio Epoca 1/10


Epoca 1/10: 100%|████████████████████████████████████████████████| 92/92 [04:13<00:00,  2.75s/batch]


Epoca 1/10, Loss: 0.3558, Tempo: 253.45s
CPU Usage: 73.6%
RAM Usage: 26.1%
GPU Memory Allocated: 0.80 GB
GPU Memory Reserved: 12.37 GB


Evaluating Model: 100%|██████████| 142/142 [00:26<00:00,  5.31it/s]


Inizio Epoca 2/10


...

Evaluating Model: 100%|██████████| 142/142 [00:26<00:00,  5.36it/s]


Inizio Epoca 10/10


Epoca 10/10: 100%|███████████████████████████████████████████████| 92/92 [04:10<00:00,  2.72s/batch]


Epoca 10/10, Loss: 0.0883, Tempo: 250.09s
CPU Usage: 73.2%
RAM Usage: 27.1%
GPU Memory Allocated: 0.80 GB
GPU Memory Reserved: 12.38 GB


Evaluating Model: 100%|██████████| 142/142 [00:26<00:00,  5.36it/s]
